In [3]:
import cv2
import face_recognition
import datetime
import random
from deepface import DeepFace

def generate_win_status(win_probability=0.3, lose_probability=0.7):
    choices = [0, 1]
    weights = [lose_probability, win_probability]
    status = random.choices(choices, weights, k=1)[0]
    if status == 1:
        return 'Congrats! You get a prize.', 'Win', 
    else:
        return "Alas! No prizes for you.", 'Loose'

def make_random_prompt_choice():
    word_descriptions = {
        "Radiant": "Glowing like a happy star.",
        "Unattractive": "Like a scarecrow in a tuxedo.",
        "Exquisite": "As fine as a snowflake's lace.",
        "Repulsive": "Beauty is in the eye of the beholder, but this one's a challenge.",
        "Angelic": "Looks straight out of heaven.",
        "Quirky": "A walking carnival of charm.",
        "Mesmerizing": "Holds you in a trance.",
        "Flawed": "Perfection's distant cousin.",
        "Alluring": "Draws admirers like a magnet.",
        "Odd": "A puzzle for the eyes.",
        "Flawless": "Smooth as polished marble.",
        "Unusual": "Stranger than a cat wearing sunglasses indoors.",
        "Enchanting": "Spells beauty with every move.",
        "Distinctive": "One of a kind, no rewind.",
        "Graceful": "Dances through life with ease.",
        "Homely": "Cozy charm that warms hearts.",
        "Stunning": "A jaw-dropping sight to behold.",
        "Plain": "Faces don't get flatter than this.",
        "Ethereal": "Seems almost otherworldly.",
        "Peculiar": "A delightful twist of the ordinary."
    }
    one_liners = list(word_descriptions.keys())
    choice = random.randint(0, len(one_liners)-1)

    one_liner = one_liners[choice]
    voice = word_descriptions[one_liner]

    return one_liner, voice

def largest_face_location(face_locations: list):
    if len(face_locations) != 0:
        largest_face = max(face_locations, key=lambda face: (face[2] - face[0]) * (face[3] - face[1])) # calculates the area of bounding box and selects max one

        return largest_face
    else:
        return None  # Return None if the list is empty

def place_in_frame(location, frame, face_location, text, font_scale=0.6, font_thickness=2, color=(0, 0, 255)):
    
    (top, right, bottom, left) = face_location
    
    bbox_center_x = (left + right) // 2
    bbox_top = top
    bbox_bottom = bottom

    # Set the font and other text parameters
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = font_scale  # Use a larger font size for better visibility
    font_thickness = font_thickness  # Use a thinner font thickness for normal weight

    # Calculate the size of the text to be drawn
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)

    if location == 'top':
        text_x = bbox_center_x - (text_size[0] // 2)
        text_y = bbox_top - 10  # You can adjust the vertical offset (e.g., 10 pixels)

        # Draw the text on the frame
        cv2.putText(frame, text, (text_x, text_y), font, font_scale, color, font_thickness)

        return frame
    
    elif location == 'bottom':
        text_x = bbox_center_x - (text_size[0] // 2)
        text_y = bbox_bottom + text_size[1] + 10  # You can adjust the vertical offset (e.g., 10 pixels)

        # Draw the text on the frame
        cv2.putText(frame, text, (text_x, text_y), font, font_scale, color, font_thickness)

        return frame

location, encoding, landmark, timing, countdown, is_prompt_set, validate_win_status = None, None, None, None, None, False, None

video_capture = cv2.VideoCapture(0)
frame_image=cv2.imread(r"Frame 2.jpg")

while True:
    ret, live_frame = video_capture.read()
    live_feed_height, live_feed_width, _ = live_frame.shape
    frame_height,frame_width,_ = frame_image.shape

    rgb_frame = live_frame[:,:,::-1]
    result_frame=frame_image.copy()
    x_offset = 66
    y_offset = 169

    face_locations = face_recognition.api.face_locations(rgb_frame) # For face detection

    if len(face_locations) != 0:

        if location == None:
            location = [largest_face_location(face_locations)]
            encoding = face_recognition.api.face_encodings(rgb_frame, location)[0]
            landmark = face_recognition.api.face_landmarks(rgb_frame, location)[0]
            timing = datetime.datetime.now()
            countdown = 5
            is_prompt_set = False

        else:
            face_encodings = face_recognition.api.face_encodings(rgb_frame, face_locations)
            comparison = face_recognition.api.compare_faces(face_encodings, encoding)
            if True not in comparison:
                location = [largest_face_location(face_locations)]
                encoding = face_recognition.api.face_encodings(rgb_frame, location)[0]
                landmark = face_recognition.api.face_landmarks(rgb_frame, location)[0]
                timing = datetime.datetime.now()
                countdown = 5
                is_prompt_set = False
            else:
                index = comparison.index(True)
                location = [face_locations[index]]
                encoding = face_encodings[index]
                landmark = face_recognition.api.face_landmarks(rgb_frame, location)[0]
                current_time = datetime.datetime.now()
                if (current_time - timing).total_seconds() >= 1:
                    timing = current_time

                    if isinstance(countdown, int) and countdown > 1:
                        countdown = countdown - 1
                    else:
                        if is_prompt_set == False:
                            is_prompt_set = True
                            win_status, validate_win_status = generate_win_status()
                            one_liner, voice_prompt = make_random_prompt_choice()
                            countdown = win_status

        (top,right,bottom,left) = location[0]
        cv2.rectangle(live_frame,(left,top),(right,bottom),(0,255,255),2)
        
        if isinstance(countdown, int):
            frame = place_in_frame(location='top', frame=frame, face_location=location[0], text=str(countdown), font_scale=3, font_thickness=2, color=(0, 255, 0))
            # cv2.putText(frame, str(countdown), (right -130, top -50), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 0, 255), 4)

        elif isinstance(countdown, str):

            cropped_frame = rgb_frame[top:bottom, left:right]
            
            # Placing text (0, 0, 255)
            if validate_win_status == 'Win':
                live_frame = place_in_frame(location='top', frame=live_frame, face_location=location[0], text=str(countdown), font_scale=0.5, font_thickness=2, color=(0, 255, 0))
            elif validate_win_status == 'Loose':
                live_frame = place_in_frame(location='top', frame=live_frame, face_location=location[0], text=str(countdown), font_scale=0.5, font_thickness=2, color=(0, 0, 255))

            # faces = DeepFace.analyze(cropped_frame, actions=['gender', 'emotion','age'], enforce_detection=False)
            # face_data =  faces[0]
            # dominant_gender = face_data['dominant_gender']
            # dominant_emotion = face_data['dominant_emotion']
            # age = face_data['age']
            # text = f"Reaction: {dominant_emotion}, Gender: {dominant_gender}, Age_guess: {age}" # Placing text

            faces = DeepFace.analyze(cropped_frame, actions=['emotion'], enforce_detection=False)
            face_data =  faces[0]
            dominant_emotion = face_data['dominant_emotion']
            text = f"Reaction: {dominant_emotion}" # Placing text

            live_frame = place_in_frame(location='bottom', frame=live_frame, face_location=location[0], text=str(text), font_scale=0.5, font_thickness=2)
    else:
        location, encoding, landmark, timing, countdown, is_prompt_set = None, None, None, None, None, False
    
    result_frame[y_offset:y_offset+live_feed_height, x_offset:x_offset+live_feed_width] = live_frame
    cv2.imshow('rgb_frame', result_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

Action: emotion: 100%|██████████| 1/1 [00:00<00:00,  5.42it/s]


In [2]:
import random

random_number = random.randint(0, 6)
print(random_number)


4


In [5]:
import random

choices = [0, 1]
weights = [0.7, 0.3]  # Corresponding probabilities: 70% for 0 and 30% for 1

random_number = random.choices(choices, weights, k=1)[0]
print(random_number)


0
